In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.info

<bound method DataFrame.info of           id  ... label
0          0  ...     1
1          1  ...     0
2          2  ...     1
3          3  ...     1
4          4  ...     1
...      ...  ...   ...
20795  20795  ...     0
20796  20796  ...     0
20797  20797  ...     0
20798  20798  ...     1
20799  20799  ...     1

[20800 rows x 5 columns]>

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df['label'].nunique()

2

In [8]:
df = df.dropna()

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [11]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [12]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [13]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


In [14]:
#vocabulary size
voc_size=5000

<h3> One Hot representation </h3>

In [15]:
message = X.copy()

In [16]:
message.reset_index(inplace=True)
#As we dropped some of the Nan values.

In [17]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\E0551793\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
ps=PorterStemmer()
corpus=[]
for i in range(len(message)):
    text = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    text = text.lower()
    text = text.split()

    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
one_hot_rep = [one_hot(word, voc_size) for word in corpus]
one_hot_rep

[[4998, 3165, 247, 4478, 378, 4514, 268, 4337, 4473, 1674],
 [3506, 298, 1019, 758, 4739, 2292, 4094],
 [3225, 649, 3981, 1834],
 [3417, 4578, 4612, 1697, 4450, 855],
 [1232, 4739, 3198, 660, 1413, 1734, 4739, 1113, 4541, 4994],
 [811,
  1998,
  3373,
  1290,
  2125,
  2788,
  2196,
  2239,
  873,
  4416,
  800,
  3089,
  3817,
  106,
  4094],
 [682, 4261, 3179, 3068, 3412, 4307, 1107, 3350, 2971, 3046, 2625],
 [986, 3687, 971, 2729, 1817, 4605, 2788, 4991, 2971, 3046, 2625],
 [3377, 1042, 190, 2075, 4249, 4895, 1438, 962, 2788, 2181],
 [1792, 4080, 80, 2811, 1746, 1546, 2475, 3251],
 [2347, 1866, 1105, 3312, 3292, 2603, 3062, 2877, 4804, 1905, 2775],
 [1697, 1183, 378, 4895, 2788, 1817],
 [2879, 3933, 529, 2197, 4881, 856, 333, 2608, 4342],
 [2001, 2672, 1896, 2125, 4638, 1104, 2493, 2971, 3046, 2625],
 [2796, 4905, 361, 357, 4668, 2971, 3046, 2625],
 [555, 448, 4909, 1769, 1041, 2706, 2304, 3048, 758, 3881],
 [367, 162, 298],
 [1168, 1828, 3048, 4616, 2788, 3842, 787, 4094],
 [789, 3

Padding

In [22]:
sent_len = 20
pad_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_len)
print(pad_docs)

[[   0    0    0 ... 4337 4473 1674]
 [   0    0    0 ... 4739 2292 4094]
 [   0    0    0 ...  649 3981 1834]
 ...
 [   0    0    0 ... 2971 3046 2625]
 [   0    0    0 ... 1914 4035  966]
 [   0    0    0 ... 3867 2674 3897]]


In [23]:
len(pad_docs)

18285

In [24]:
feature_dim = 40
model = Sequential()
model.add(Embedding(voc_size, feature_dim, input_length = sent_len))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\FakeNewsClassifier-LSTM\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
import numpy as np
X_final = np.array(pad_docs)
y_final = np.array(y)

In [27]:
len(pad_docs), y.shape

(18285, (18285,))

In [28]:
X_final.shape

(18285, 20)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [30]:
y_test

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10


192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.7702 - loss: 0.4340 - val_accuracy: 0.9109 - val_loss: 0.2199
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9439 - loss: 0.1443 - val_accuracy: 0.9203 - val_loss: 0.1907
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9628 - loss: 0.1044 - val_accuracy: 0.9143 - val_loss: 0.2064
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9724 - loss: 0.0776 - val_accuracy: 0.9165 - val_loss: 0.2143
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9798 - loss: 0.0610 - val_accuracy: 0.9158 - val_loss: 0.2884
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9841 - loss: 0.0490 - val_accuracy: 0.9185 - val_loss: 0.3011
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9874 - loss: 0.0392 - val_accuracy: 0.9147 - val_loss: 0.3127
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9925 - loss: 0.0255 - val_accuracy: 0.91

In [32]:
# tf.keras.utils.plot_model(model)

AttributeError: module 'pydot' has no attribute 'InvocationException'

Performance Metrics

In [33]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
# y_pred = np.argmax(y_pred_value,axis=1)

from sklearn.metrics import confusion_matrix

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [34]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [35]:
confusion_matrix(y_test, y_pred)

array([[3121,  298],
       [ 190, 2426]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9191383595691798

Make Predictions

In [37]:
test_df = pd.read_csv("test.csv")

In [38]:
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [39]:
test_df.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [40]:
test_df = test_df.dropna()

In [41]:
test_message = test_df.copy()
test_message.reset_index(inplace=True)

In [42]:
ps=PorterStemmer()
corpus=[]
for i in range(len(test_message)):
    text = re.sub('[^a-zA-Z]', ' ', test_message['title'][i])
    text = text.lower()
    text = text.split()

    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [43]:
corpus

['specter trump loosen tongu purs string silicon valley new york time',
 'nodapl nativ american leader vow stay winter file lawsuit polic',
 'tim tebow attempt anoth comeback time basebal new york time',
 'keiser report meme war e',
 'pelosi call fbi investig find russian donald trump breitbart',
 'weekli featur profil randi shannon',
 'u gener admir endors trump command chief',
 'work class hero john brennon',
 'rise mandatori vaccin mean end medic freedom',
 'communist terror small busi',
 'comput programm come forward admit paid rig vote booth trump right usa newsflash',
 'thiev take chunk chang pound berlin museum new york time',
 'new england patriot owner still sore n f l payback sight new york time',
 'colleg republican yaf sue berkeley ann coulter event breitbart',
 'trump melt accus us postal servic steal elect clinton',
 'visit madagascar leav red swimsuit lemur recip home new york time',
 'rees peanut butter cup cheap full toxic chemic',
 'presid obama presid elect donald tr

In [44]:
one_hot_rep = [one_hot(word, voc_size) for word in corpus]
one_hot_rep

[[3725, 2788, 2558, 4704, 1331, 4065, 4026, 3160, 2971, 3046, 2625],
 [553, 1415, 376, 3285, 627, 1176, 1299, 656, 3657, 602],
 [3739, 215, 3073, 3523, 2672, 2625, 1869, 2971, 3046, 2625],
 [523, 3403, 2504, 830, 4848],
 [3509, 3117, 367, 1324, 4638, 2347, 4616, 2788, 4094],
 [3957, 640, 2886, 3568, 828],
 [1531, 4268, 100, 2396, 2788, 4158, 361],
 [4290, 3751, 907, 3558, 4814],
 [2816, 1706, 3114, 4522, 3250, 729, 1782],
 [1494, 4542, 4163, 1473],
 [2379, 3950, 4234, 113, 1796, 3933, 4620, 1488, 4397, 2788, 1986, 1291, 1486],
 [3065, 922, 2184, 2559, 1837, 1616, 728, 2971, 3046, 2625],
 [2971,
  3860,
  4488,
  2404,
  1159,
  1265,
  3572,
  2674,
  967,
  802,
  1079,
  2971,
  3046,
  2625],
 [936, 3430, 4602, 1459, 2624, 3727, 1713, 4036, 4094],
 [2788, 4498, 1372, 1697, 96, 4214, 4081, 1825, 1019],
 [1287, 4172, 17, 1488, 2838, 2229, 1182, 1104, 2971, 3046, 2625],
 [2574, 2716, 2166, 1457, 178, 874, 4360, 2044],
 [1135, 3377, 1135, 1825, 4616, 2788, 4530, 3968, 4998],
 [2811, 690

In [45]:
pad_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_len)
print(pad_docs)

[[   0    0    0 ... 2971 3046 2625]
 [   0    0    0 ...  656 3657  602]
 [   0    0    0 ... 2971 3046 2625]
 ...
 [   0    0    0 ... 2971 3046 2625]
 [   0    0    0 ... 2971 3046 2625]
 [   0    0    0 ... 2971 3046 2625]]


In [46]:
X_test = np.array(pad_docs)

In [47]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [48]:
predictions = [int(np.round(pred[0])) for pred in y_pred]
predictions

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,


In [49]:
submission = pd.DataFrame({
    "id" : test_df['id'],
    "label" : predictions
})
submission

,id,label
0,20800,0
2,20802,0
3,20803,0
4,20804,1
6,20806,0
...,...,...
5194,25994,0
5195,25995,0
5196,25996,0
5197,25997,0


In [51]:
submission.to_csv('submission1.csv', index=False)